## Step2 with ErgoSCF

In [1]:
import os
import sys

# Fisrt, we add the location of the library to test to the PYTHON path
if sys.platform=="cygwin":
    from cyglibra_core import *
elif sys.platform=="linux" or sys.platform=="linux2":
    from liblibra_core import *
import util.libutil as comn
from libra_py import ERGO_methods
from libra_py import units
from libra_py.workflows.nbra import step2_ergoscf


/home/alexey/miniconda2/envs/libra/lib/python2.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: to-Python converter for std::vector<std::complex<double>, std::allocator<std::complex<double> > > already registered; second conversion method ignored.
  
/home/alexey/miniconda2/envs/libra/lib/python2.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: to-Python converter for std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > > already registered; second conversion method ignored.
  
/home/alexey/miniconda2/envs/libra/lib/python2.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: to-Python converter for boost::python::detail::container_element<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > >, unsigned long, boost::python::detail::final_vector_derived_policies<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator

Define the function to run the ErgosCF calculations for each geometry. This function essentially defines the core of the input file to the ErgoSCF code. Only the coordinates and the name of the executable may be changed.

Note the "Angstrom" keyword following the "molecule_inline" - it indicates that the input is provided in Angstrom units.

If your input (e.g. in the xyz trajectory file) is in the a.u. (Bohr) units, just remove the "Angstrom" keyword

In [2]:
def run(EXE, COORDS):
    inp = """#!bin/sh
%s << EOINPUT > /dev/null
spin_polarization = 0
molecule_inline Angstrom
%sEOF
basis = "STO-3G"
use_simple_starting_guess=1
scf.create_mtx_files_F = 1
scf.create_mtx_file_S = 1
XC.sparse_mode = 1
run "LDA"
EOINPUT
""" % (EXE, COORDS)
    return inp

Also define how to run the calculations for the "weird" geometry, when two snapshots are overlapped and form a "super-molecule". In this case, we don't really need to print out the Fock matrices

In [3]:
def run1(EXE, COORDS):
    inp = """#!bin/sh
%s << EOINPUT > /dev/null
spin_polarization = 0
molecule_inline Angstrom
%sEOF
basis = "STO-3G"
use_simple_starting_guess=1
scf.create_mtx_file_S = 1
scf.create_mtx_files_S_and_quit = 1
XC.sparse_mode = 1
run "LDA"
EOINPUT
""" % (EXE, COORDS)
    return inp

Ok, so now we can run the step2 calculations! Just create the "res" dir, since this is where the results will be printed out

In [4]:
params = {"EXE":"/mnt/c/cygwin/home/Alexey-user/Soft/ergo-3.7/source/ergo", "md_file":"md1.xyz",
          "isnap":0, "fsnap":5, "dt": 1.0 * units.fs2au
         }

os.system("mkdir res")
step2_ergoscf.run_step2(params, run, run1)